### Text Analysis Procedure

The text analysis will identify the words in the business functions for the listed Google acquisitions vs. acquisitions of other similar companies (e.g. Facebook, Apple). This will create a bag-of-words model that will be used to examine the profiles of the select public companies and determine a similiarity index that will be used as a probability measure for how likely they would be acquired by Alphabet. 

Following this, the the IPO list will be divided into the leaf nodes identified in the decision tree model, from which point the average probabilties will be calculated, and a shortlist of the top 10 companies will be identified from here as the most likely to be acquired by Google as per this model.   

In [1]:
conda install -c anaconda beautifulsoup4

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import bs4
import requests
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as bsoup

In [3]:
import numpy as np
import pandas as pd

### Data Cleaning & Scraping 

The retrieved dataset with the list of public companies has some faults in it. Some of these firms are no longer public, others are located on stock exchanges that are less mainstream - making retrieving further data on them more difficult (this project's focus will be on the NYSE and the NASDAQ Global Select - the highest tier on the NASDAQ). Therefore, cleaning this dataset is the first step before we can obtain more information.

In [6]:
ipo_links = pd.read_csv('Final Project/IPO Calendar.csv')
ipos_list = ipo_links[ipo_links['Exchange'].isin(["NASDAQ Global Select", "NYSE"])]
ipos_list['Symbol'][12] = "PACK" #updated ticker
ipos_list['Symbol'][16] = "NEX"  #updated ticker
ipos_list['Symbol'][82] = "RMG"  #updated ticker
ipos_list['Symbol'][98] = "LHC"  #updated ticker
ipos_list['Symbol'][119] = "SHLL"  #updated ticker
ipos_list['Symbol'][143] = "TRNE"  #updated ticker
ipos_list['Symbol'][193] = "RPLA"  #updated ticker
ipos_list['Symbol'][204] = "ALTG"  #updated ticker
ipos_list['Symbol'][290] = "MGY"  #updated ticker
ipos_list['Symbol'][332] = "IR"  #updated ticker
ipos_list['Symbol'][397] = "GIX"  #updated ticker
ipos_list['Symbol'][403] = "VRT"  #updated ticker
ipos_list['Symbol'][414] = "FPAC"  #updated ticker
ipos_list['Symbol'][437] = "RPAY"  #updated ticker
ipos_list['Symbol'][492] = "CCX"  #updated ticker
ipos_list['Symbol'][495] = "ACEL"  #updated ticker
ipos_list['Symbol'][500] = "NFH"  #updated ticker
ipos_list['Symbol'][526] = "PIC"  #updated ticker
ipos_list['Symbol'][527] = "SCPE"  #updated ticker
ipos_list['Symbol'][545] = "OAC"  #updated ticker
ipos_list['Symbol'][595] = "SBE"  #updated ticker
ipos_list['Symbol'][654] = "SPAQ"  #updated ticker
ipos_list['Symbol'][670] = "NSCO"  #updated ticker
ipos_list['Symbol'][679] = "MFAC"  #updated ticker
ipos_list['Symbol'][698] = "SPCE"  #updated ticker
ipos_list['Symbol'][723] = "BEST"  #updated ticker
ipos_list['Symbol'][784] = "CCH"  #updated ticker
ipos_list['Symbol'][829] = "GLEO"  #updated ticker
ipos_list['Symbol'][842] = "VVNT"  #updated ticker
ipos_list['Symbol'][914] = "JIH"  #updated ticker
ipos_list['Symbol'][950] = "LGC"  #updated ticker
ipos_list['Symbol'][990] = "KLR"  #updated ticker
ipos_list = ipos_list.drop([27, 28, 48, 49, 54, 117, 135, 149, 208, 237, 281, 287, 486, 602, 626, 683, 759, 763, 790, 821, 854, 879, 889, 898, 923, 937, 973, 979, 987, 1007, 1011], axis=0) #they went private/bankrupt/not on the same exchange
ipos_list.reset_index(drop=True, inplace=True)
ipos_list

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

,Symbol,Company Name,Exchange,Price,Shares,Date,Offer Amount,Actions
0,ILPT,Industrial Logistics Properties Trust,NASDAQ Global Select,24.00,"20,000,000",01/12/2018,480000000,Priced
1,LBRT,Liberty Oilfield Services Inc.,NYSE,17.00,"12,731,092",01/12/2018,216428564,Priced
2,PACK,Ranpak Holdings Corp.,NYSE,10.00,"30,000,000",01/18/2018,300000000,Priced
3,NINE,"Nine Energy Service, Inc.",NYSE,23.00,"7,000,000",01/19/2018,161000000,Priced
4,ADT,ADT Inc.,NYSE,14.00,"105,000,000",01/19/2018,1470000000,Priced
...,...,...,...,...,...,...,...,...
394,CASA,Casa Systems Inc,NASDAQ Global Select,13.00,"6,000,000",12/15/2017,78000000,Priced
395,NMRK,"NEWMARK GROUP, INC.",NASDAQ Global Select,14.00,"20,000,000",12/15/2017,280000000,Priced
396,TRVG,trivago N.V.,NASDAQ Global Select,11.00,"26,110,118",12/16/2016,287211298,Priced
397,AMTB,Amerant Bancorp Inc.,NASDAQ Global Select,13.00,"6,300,000",12/19/2018,81900000,Priced


In [176]:
profile_file = "profiles.csv"
f = open(profile_file, "w")
header = "Description\n"
f.write(header)

def ipo_profile(ticker):
    ipo1_url = f'https://finance.yahoo.com/quote/{ticker}/profile?p={ticker}'
    ipo_Client1 = uReq(ipo1_url)
    ipo1_html = ipo_Client1.read()
    ipo_Client1.close()

    ipo1_soup = bsoup(ipo1_html, "html.parser")
    ipo1_prfl = ipo1_soup.find('p', {'class':'Mt(15px) Lh(1.6)'}).text
    if type(ipo1_prfl) == None:
        return print("")
    else:
        return ipo1_prfl

symbols = ipos_list['Symbol']
for symbol in symbols:
    #print(ipo_profile(symbol))
    f.write(ipo_profile(symbol).replace(",","") + "\n")

f.close()


ILPT is a real estate investment trust, or REIT, that owns and leases industrial and logistics properties throughout the United States. ILPT is managed by the operating subsidiary of The RMR Group Inc. (Nasdaq: RMR), an alternative asset management company that is headquartered in Newton, MA.
Liberty Oilfield Services Inc. provides hydraulic fracturing services to onshore oil and natural gas exploration and production companies in North America. The company offers its services primarily in the Permian Basin, the Eagle Ford Shale, the Denver-Julesburg Basin, the Williston Basin, and the Powder River Basin. Liberty Oilfield Services Inc. was founded in 2011 and is headquartered in Denver, Colorado.
Ranpak Holdings Corp. and its subsidiaries provide product protection solutions for e-commerce and industrial supply chains. The company manufactures and assembles proprietary protective systems that convert kraft paper into a range of packaging and cushioning products to address its customers

Gates Industrial Corporation plc manufactures and sells engineered power transmission and fluid power solutions worldwide. The company offers synchronous or asynchronous belts, including V-belts, CVT belts, and Micro-V belts, as well as related components, such as sprockets, pulleys, water pumps, tensioners, or other accessories; and metal drive components and kits for stationary and mobile drives, engine systems, personal mobility, and vertical lifts application platforms. It also provides fluid power products comprising hydraulic hoses, tubing, and fittings, as well as assemblies; and engine and industrial hoses for use in stationary hydraulics, mobile hydraulics, engine systems, and other industrial applications. The company serves various end markets comprising the construction, agriculture, energy, automotive, transportation, general industrial, consumer products, and others. It sells its engineered products under the Gates brand. The company offers its products to replacement cha

Laureate Education, Inc., together with its subsidiaries, provides higher education programs and services to students through a network of universities and higher education institutions. It offers a range of undergraduate and graduate degree programs primarily in the areas of business and management, medicine and health sciences, and engineering and information technology through campus-based, online, and hybrid programs. The company also operates online institutions that offer professional degree programs primarily for the working adults with undergraduate and graduate degree programs. It provides its services in Brazil, Mexico, Chile, Peru, Australia, China, New Zealand, the United Kingdom, and the United States. The company was formerly known as Sylvan Learning Systems, Inc. and changed its name to Laureate Education, Inc. in May 2004. The company was founded in 1989 and is headquartered in Baltimore, Maryland.
Invitation Homes is the nation's premier single-family home leasing comp

Huami Corporation, a biometric and activity data-driven company, develops, manufactures, and sells smart wearable technological devices in the People's Republic of China. It operates through two segments, Xiaomi Wearable Products, and Self-Branded Products and Others. The company offers smart bands, watches, and scales; and a range of accessories, including bands, watch straps, earphones, sportswear, home gym, treadmill, etc. under the Xiaomi and Amazfit brands. It provides charts and graphs to display analysis of the activity and biometric data collected from users through its Mi Fit and Amazfit mobile apps. Huami Corporation has strategic collaborations with Timex Group to develop smart watches; and AliveCor, Inc. to deliver a medical functionality to wearable devices. The company was founded in 2013 and is headquartered in Hefei, the People's Republic of China.
Gossamer Bio, Inc., a clinical-stage biopharmaceutical company, focuses on discovering, acquiring, developing, and commerci

Hamilton Lane Incorporated is an investment firm specializing in direct and fund of fund investments. It provides following services: separate accounts (customized to each individual client and structured as single client vehicles); specialized strategies (fund-of-funds, secondaries, co-investments, taft-hartley, distribution management); advisory relationships (including due diligence, strategic portfolio planning, monitoring and reporting services); and reporting and analytics solutions. For direct investments, the firm invests in mid and late venture, mature companies, growth equity, emerging growth, distressed debt, later stage, turnarounds, bridge financing, mezzanine financing, and buyouts in middle market companies. For fund of fund investments, it invests in mezzanine, venture capital, private equity, turnaround, secondary investments, real estate, and special situation funds. The firm invests in real estate investments. It also invest in technology, healthcare, education, natu

...

Evoqua Water Technologies Corp. provides a range of water and wastewater treatment systems and technologies, and mobile and emergency water supply solutions and services for industrial, commercial, and municipal water treatment markets. It operates in two segments, Integrated Solutions and Services, and Applied Product Technologies. The Integrated Solutions and Services segment offers capital systems and related recurring aftermarket services, parts, and consumables, as well as long-term and short-term service contracts, and emergency services for treating industrial process water, utility water, and wastewater. It serves customers in the pharmaceutical, food and beverage, power, and chemical processing industries. The Applied Product Technologies segment offers advanced filtration and separation products, such as Memcor membranes, Ionpure technologies, and Vortisand systems; wastewater treatment technologies, including the BioMag systems, clarification systems, and odor control and sl

Juniper Industrial Holdings, Inc. does not have significant operations. The company intends to effect into a merger, capital stock exchange, asset acquisition, stock purchase, reorganization, or similar business combination with one or more businesses in the industrial sector. The company was founded in 2019 and is based in Chatham, New Jersey.
Apellis Pharmaceuticals, Inc., a clinical-stage biopharmaceutical company, focuses on the development of therapeutic compounds through the inhibition of the complement system for autoimmune and inflammatory diseases. Its lead product candidate is APL-2 that is in Phase III clinical trials for the treatment of geographic atrophy in age-related macular degeneration and paroxysmal nocturnal hemoglobinuria diseases; and in Phase II clinical trials for the treatment of cold agglutinin and warm antibody autoimmune hemolytic anemia diseases, as well as in Phase II clinical trials to treat four types of glomerular diseases, such as C3 glomerulopathy, Ig

SailPoint Technologies Holdings, Inc. designs, develops, and markets identity governance software solutions in the United States, Europe, the Middle East, Africa, and internationally. The company offers software and software as a service solutions, which help organizations to govern the digital identities of employees, contractors, business partners, software bots, and other human and non-human users, as well as manage their constantly changing access rights to enterprise applications and data across hybrid IT environments, spanning on-premises, cloud and mobile applications, and file storage platforms. Its solutions include IdentityIQ, an on-premises identity governance solution; IdentityNow, a cloud-based multi-tenant governance platform; and IdentityAI, a multi-tenant AI and ML SaaS subscription offering that helps organizations detect potential threats before they turn into security breaches. The company sells its products and solutions to commercial enterprises, financial institut

Luther Burbank Corporation operates as the bank holding company for Luther Burbank Savings that provides various banking products and services for real estate investors, professionals, entrepreneurs, high net worth individuals, and commercial businesses. The company offers interest and noninterest-bearing transaction accounts, certificates of deposit, and money market accounts. It also provides commercial real estate loans, including first mortgage loans for the purchase, refinance, or build-out of tenant improvements on investor owned multifamily residential properties, as well as loans for the purchase, refinance, or improvement of office, retail, and light industrial properties; single family residential loans; and mortgage products, such as a portfolio of 30-year fixed rate first mortgage and a forgivable second mortgage. In addition, the company offers ATM, debit cards, and online and mobile banking services; engages in the real estate investment; and issues trust preferred securi

Newmark Group, Inc. provides commercial real estate services in the United States and internationally. The company's investor/owner services and products include capital markets, such as investment sales; and agency leasing, property management, valuation and advisory, and diligence and underwriting, as well as government sponsored enterprise lending, loan servicing, debt and structured finance, loan sales, mortgage broking and equity-raising under the Newmark Knight Frank name. It occupier services and products comprise tenant representation, real estate management technology systems, workplace and occupancy strategy, global corporate services consulting, project management, lease administration, and facilities management. The company provides its services to commercial real estate tenants, owner-occupiers, investors, and developers. As of February 27, 2020, it operated approximately 480 offices in 6 continents. The company was formerly known as Newmark Knight Frank and changed its na

In [7]:
company_desc = pd.read_csv("Final Project/profiles.csv")
ipos_list['Description'] = company_desc
ipos_list[70:80]

,Symbol,Company Name,Exchange,Price,Shares,Date,Offer Amount,Actions,Description
70,SNDR,"Schneider National, Inc.",NYSE,19.0,"28,947,000",04/06/2017,549993000,Priced,Schneider National Inc. a transportation and l...
71,ELVT,"Elevate Credit, Inc.",NYSE,6.5,"12,400,000",04/06/2017,80600000,Priced,Elevate Credit Inc. provides online credit sol...
72,OKTA,"Okta, Inc.",NASDAQ Global Select,17.0,"11,000,000",04/07/2017,187000000,Priced,Okta Inc. provides identity management platfor...
73,ALTG,ALTA EQUIPMENT GROUP INC.,NYSE,10.0,"12,500,000",04/09/2019,125000000,Priced,Alta Equipment Group Inc. owns and operates in...
74,TUFN,Tufin Software Technologies Ltd.,NYSE,14.0,"7,700,000",04/11/2019,107800000,Priced,Tufin Software Technologies Ltd. develops mark...
75,PD,"PagerDuty, Inc.",NYSE,24.0,"9,070,000",04/11/2019,217680000,Priced,PagerDuty Inc. operates a platform for real-ti...
76,ZUO,ZUORA INC,NYSE,14.0,"11,000,000",04/12/2018,154000000,Priced,Zuora Inc. provides a cloud-based software on ...
77,JMIA,Jumia Technologies AG,NYSE,14.5,"13,500,000",04/12/2019,195750000,Priced,Jumia Technologies AG operates an e-commerce p...
78,TOCA,Tocagen Inc,NASDAQ Global Select,10.0,"8,500,000",04/13/2017,85000000,Priced,Tocagen Inc. a clinical-stage cancer-selective...
79,CADE,Cadence Bancorporation,NYSE,20.0,"7,500,000",04/13/2017,150000000,Priced,Cadence Bancorporation a financial holding com...


Now we run a program to obtain the company profiles, as well as any other information that could be useful. This data will be appended to our ipos_list dataset.

In [ ]:
country_file = "countries.csv"
f = open(country_file, "w")
header1 = "Country\n"
f.write(header1)

def ipo_country(ticker):
    ipo2_url = f'https://finance.yahoo.com/quote/{ticker}/profile?p={ticker}'
    ipo_Client2 = uReq(ipo2_url)
    ipo2_html = ipo_Client2.read()
    ipo_Client2.close()

    ipo2_soup = bsoup(ipo2_html, "html.parser")
    ipo2_cntry = ipo2_soup.find('p', {'class':'D(ib) W(47.727%) Pend(40px)'}).text
    if type(ipo2_cntry) == None:
        return print("")
    else:
        return ipo2_cntry

symbols1 = ipos_list['Symbol']
for symbol in symbols1:
    print(ipo_country(symbol))
    f.write(ipo_country(symbol).replace(",","") + "\n")

f.close()

        
        

Two Newton PlaceSuite 300 255 Washington StreetNewton, MA 02458United States617-219-1460http://www.ilptreit.com
950 17th StreetSuite 2400Denver, CO 80202United States303 515 2800http://www.libertyfrac.com
7990 Auburn RoadConcord Township, OH 44077United States440 354 4445http://www.ranpak.com
2001 Kirby DriveSuite 200Houston, TX 77019United States281 730 5100http://nineenergyservice.com
1501 Yamato RoadBoca Raton, FL 33431United States561 988 3600http://www.adt.com
South TowerSuite 600 10 Glenlake ParkwayAtlanta, GA 30328United States678-441-1400http://www.americold.com
3990 Rogerdale RoadHouston, TX 77042United States713 325 6000http://nextierofs.com
Avenida Brigadeiro Faria Lima, 13844º Andar Parte ASao Paulo, SP 01451-001Brazil55 11 3038 8127http://www.pagseguro.uol.com.br
1144 Fifteenth StreetDenver, CO 80202United States303 744 1911http://www.gates.com
520 U.S. Highway 22Suite 305Bridgewater, NJ 08807United States800-775-7936http://www.menlotherapeutics.com
Chemin des Aulx, 12Plan

In [8]:
cntry = pd.read_csv('Final Project/countries.csv')
ipos_list['Country'] = cntry['Actual Country']
ipos_list

,Symbol,Company Name,Exchange,Price,Shares,Date,Offer Amount,Actions,Description,Country
0,ILPT,Industrial Logistics Properties Trust,NASDAQ Global Select,24.00,"20,000,000",01/12/2018,480000000,Priced,ILPT is a real estate investment trust or REIT...,United States
1,LBRT,Liberty Oilfield Services Inc.,NYSE,17.00,"12,731,092",01/12/2018,216428564,Priced,Liberty Oilfield Services Inc. provides hydrau...,United States
2,PACK,Ranpak Holdings Corp.,NYSE,10.00,"30,000,000",01/18/2018,300000000,Priced,Ranpak Holdings Corp. and its subsidiaries pro...,United States
3,NINE,"Nine Energy Service, Inc.",NYSE,23.00,"7,000,000",01/19/2018,161000000,Priced,Nine Energy Service Inc. operates as an onshor...,United States
4,ADT,ADT Inc.,NYSE,14.00,"105,000,000",01/19/2018,1470000000,Priced,ADT Inc. provides security and automation solu...,United States
...,...,...,...,...,...,...,...,...,...,...
394,CASA,Casa Systems Inc,NASDAQ Global Select,13.00,"6,000,000",12/15/2017,78000000,Priced,Casa Systems Inc. provides software-centric br...,United States
395,NMRK,"NEWMARK GROUP, INC.",NASDAQ Global Select,14.00,"20,000,000",12/15/2017,280000000,Priced,Newmark Group Inc. provides commercial real es...,United States
396,TRVG,trivago N.V.,NASDAQ Global Select,11.00,"26,110,118",12/16/2016,287211298,Priced,trivago N.V. together with its subsidiaries op...,Germany
397,AMTB,Amerant Bancorp Inc.,NASDAQ Global Select,13.00,"6,300,000",12/19/2018,81900000,Priced,Amerant Bancorp Inc. operates as the bank hold...,United States


In [227]:
sector_file = "sectors.csv"
f = open(sector_file, "w")
header2 = "Sector\n"
f.write(header2)

def ipo_sector(ticker):
    ipo3_url = f'https://finance.yahoo.com/quote/{ticker}/profile?p={ticker}'
    ipo_Client3 = uReq(ipo3_url)
    ipo3_html = ipo_Client3.read()
    ipo_Client3.close()

    ipo3_soup = bsoup(ipo3_html, "html.parser")
    ipo3_sctr = ipo3_soup.findAll('span', {'class':'Fw(600)'})
    if type(ipo3_sctr[1]) == None:
        return print("")
    else:
        return ipo3_sctr[1].text
    
symbols2 = ipos_list['Symbol']
for symbol in symbols2:
    #print(ipo_sector(symbol))
    f.write(ipo_sector(symbol).replace(",","") + "\n")

f.close()

In [9]:
sctr = pd.read_csv('Final Project/sectors.csv')
ipos_list['Group'] = sctr['Group']
ipos_list

,Symbol,Company Name,Exchange,Price,Shares,Date,Offer Amount,Actions,Description,Country,Group
0,ILPT,Industrial Logistics Properties Trust,NASDAQ Global Select,24.00,"20,000,000",01/12/2018,480000000,Priced,ILPT is a real estate investment trust or REIT...,United States,Misc.
1,LBRT,Liberty Oilfield Services Inc.,NYSE,17.00,"12,731,092",01/12/2018,216428564,Priced,Liberty Oilfield Services Inc. provides hydrau...,United States,Misc.
2,PACK,Ranpak Holdings Corp.,NYSE,10.00,"30,000,000",01/18/2018,300000000,Priced,Ranpak Holdings Corp. and its subsidiaries pro...,United States,Misc.
3,NINE,"Nine Energy Service, Inc.",NYSE,23.00,"7,000,000",01/19/2018,161000000,Priced,Nine Energy Service Inc. operates as an onshor...,United States,Misc.
4,ADT,ADT Inc.,NYSE,14.00,"105,000,000",01/19/2018,1470000000,Priced,ADT Inc. provides security and automation solu...,United States,Devices
...,...,...,...,...,...,...,...,...,...,...,...
394,CASA,Casa Systems Inc,NASDAQ Global Select,13.00,"6,000,000",12/15/2017,78000000,Priced,Casa Systems Inc. provides software-centric br...,United States,Media
395,NMRK,"NEWMARK GROUP, INC.",NASDAQ Global Select,14.00,"20,000,000",12/15/2017,280000000,Priced,Newmark Group Inc. provides commercial real es...,United States,Misc.
396,TRVG,trivago N.V.,NASDAQ Global Select,11.00,"26,110,118",12/16/2016,287211298,Priced,trivago N.V. together with its subsidiaries op...,Germany,Search
397,AMTB,Amerant Bancorp Inc.,NASDAQ Global Select,13.00,"6,300,000",12/19/2018,81900000,Priced,Amerant Bancorp Inc. operates as the bank hold...,United States,Organization


In [258]:
ind_file = "industry.csv"
f = open(ind_file, "w")
header3 = "Industry\n"
f.write(header3)

def ipo_industry(ticker):
    ipo4_url = f'https://finance.yahoo.com/quote/{ticker}/profile?p={ticker}'
    ipo_Client4 = uReq(ipo4_url)
    ipo4_html = ipo_Client4.read()
    ipo_Client4.close()

    ipo4_soup = bsoup(ipo4_html, "html.parser")
    ipo4_ind = ipo4_soup.find('span', {'class':'Fw(600)'})
    if type(ipo4_ind) == None:
        return print("")
    else:
        return ipo4_ind.text

ipos_lista = ipos_list.drop([78], axis=0) 

    
symbols3 = ipos_lista['Symbol']
for symbol in symbols3:
    #print(ipo_industry(symbol))
    f.write(ipo_industry(symbol).replace(",","") + "\n")

f.close()

In [10]:
indt = pd.read_csv('Final Project/industry.csv')
ipos_list['Industry'] = indt
ipos_list

,Symbol,Company Name,Exchange,Price,Shares,Date,Offer Amount,Actions,Description,Country,Group,Industry
0,ILPT,Industrial Logistics Properties Trust,NASDAQ Global Select,24.00,"20,000,000",01/12/2018,480000000,Priced,ILPT is a real estate investment trust or REIT...,United States,Misc.,Real Estate
1,LBRT,Liberty Oilfield Services Inc.,NYSE,17.00,"12,731,092",01/12/2018,216428564,Priced,Liberty Oilfield Services Inc. provides hydrau...,United States,Misc.,Energy
2,PACK,Ranpak Holdings Corp.,NYSE,10.00,"30,000,000",01/18/2018,300000000,Priced,Ranpak Holdings Corp. and its subsidiaries pro...,United States,Misc.,Consumer Cyclical
3,NINE,"Nine Energy Service, Inc.",NYSE,23.00,"7,000,000",01/19/2018,161000000,Priced,Nine Energy Service Inc. operates as an onshor...,United States,Misc.,Energy
4,ADT,ADT Inc.,NYSE,14.00,"105,000,000",01/19/2018,1470000000,Priced,ADT Inc. provides security and automation solu...,United States,Devices,Industrials
...,...,...,...,...,...,...,...,...,...,...,...,...
394,CASA,Casa Systems Inc,NASDAQ Global Select,13.00,"6,000,000",12/15/2017,78000000,Priced,Casa Systems Inc. provides software-centric br...,United States,Media,Technology
395,NMRK,"NEWMARK GROUP, INC.",NASDAQ Global Select,14.00,"20,000,000",12/15/2017,280000000,Priced,Newmark Group Inc. provides commercial real es...,United States,Misc.,Real Estate
396,TRVG,trivago N.V.,NASDAQ Global Select,11.00,"26,110,118",12/16/2016,287211298,Priced,trivago N.V. together with its subsidiaries op...,Germany,Search,Communication Services
397,AMTB,Amerant Bancorp Inc.,NASDAQ Global Select,13.00,"6,300,000",12/19/2018,81900000,Priced,Amerant Bancorp Inc. operates as the bank hold...,United States,Organization,Financial Services


In [11]:
ipos_csv = ipos_list.copy()
ipos_csv.to_csv('Final Project/IPO Information.csv')

### Text Analysis - Bag-of-words  

Now that we have the information that we need, we can move to creating the bag-of-words model.

In [12]:
from sklearn.feature_extraction import text  
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
acqx = pd.read_csv('Final Project/acquired companies.csv')
acqx['Google'] = np.where(acqx['ParentCompany'] == 'Alphabet', 1, 0) #binary identifier that will be used to train the model
acqx

,Acquisition Date,Company,Business,Country,Price,Used as or integrated with:,Group,ParentCompany,Google
0,2005,Systems Research & Development,Identity management,United States,NaN,NaN,NaN,IBM,0
1,2005,en'tegrate,Software,United States,NaN,NaN,NaN,Microsoft,0
2,2005,Stadeon,Online game,United States,NaN,Yahoo! Games,NaN,Yahoo,0
3,2005,Corio,Application Services,United States,1.820000e+08,NaN,NaN,IBM,0
4,2005,Groove Networks,Community software,United States,NaN,NaN,NaN,Microsoft,0
...,...,...,...,...,...,...,...,...,...
668,01-Nov-19,Fitbit,Wearables,United States,2.100000e+09,NaN,NaN,Alphabet,1
669,18-Nov-19,CloudSimple,Cloud hosting,United States,0.000000e+00,Google Cloud Platform,Productivity,Alphabet,1
670,19-Dec-19,Typhoon Studios,Video game development,Canada,0.000000e+00,Stadia,Media,Alphabet,1
671,14-Jan-20,Pointy,Local retail inventory feeds,Ireland,1.630000e+08,Google Maps,Search,Alphabet,1


In [13]:
bustxt_bag = text.CountVectorizer(stop_words="english")
bustxt_bag.fit(acqx['Business'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [14]:
all_bustxt = bustxt_bag.get_feature_names()
len(all_bustxt)

543

In [15]:
all_bustxt[1:50]

['3d',
 '64',
 'access',
 'ad',
 'ads',
 'advanced',
 'advertiser',
 'advertising',
 'aerial',
 'agency',
 'aggregator',
 'ai',
 'airborne',
 'ajax',
 'altitude',
 'american',
 'amp',
 'analysis',
 'analytics',
 'android',
 'anonymous',
 'answer',
 'anti',
 'api',
 'app',
 'appliance',
 'application',
 'applications',
 'apps',
 'architecture',
 'arms',
 'artificial',
 'asset',
 'assets',
 'assistant',
 'associated',
 'atlas',
 'attribution',
 'audio',
 'augmented',
 'authentication',
 'automated',
 'automatic',
 'automation',
 'backends',
 'backup',
 'based',
 'behavioral',
 'beta']

In [16]:
y = acqx['Google']
X = bustxt_bag.transform(acqx['Business'])
X

<673x543 sparse matrix of type '<class 'numpy.int64'>'
	with 1714 stored elements in Compressed Sparse Row format>

In [17]:
Goog_nb = MultinomialNB()
Goog_nb.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [18]:
Goog_nb.score(X,y)

0.8216939078751857

After initializing the model, we apply it to the company list dataset to obtain the probabilties, denoted in the Google_prob column.

In [19]:
bagged = bustxt_bag.transform(ipos_list['Description'])
probas = Goog_nb.predict_proba(bagged)[:,1]
ipos_list['Google_prob'] = probas
#ipos_list[ipos_list['Google_prob'].isin([1])]
#ipos_list['Google_prob'] = (ipos_list['Google_prob']*100).astype(str) + '%'
group_index = {'Search':0, 'Media':0, 'Business':0, 'OS':1, 'Organization':1, 'Productivity':1, 'Connectivity':1, 'Devices':2, 'Research':2, 'Innovation':2, 'Misc.':2}
ipos_list['Group_Index'] = ipos_list['Group'].map(group_index)

ipos_list



,Symbol,Company Name,Exchange,Price,Shares,Date,Offer Amount,Actions,Description,Country,Group,Industry,Google_prob,Group_Index
0,ILPT,Industrial Logistics Properties Trust,NASDAQ Global Select,24.00,"20,000,000",01/12/2018,480000000,Priced,ILPT is a real estate investment trust or REIT...,United States,Misc.,Real Estate,9.866884e-02,2
1,LBRT,Liberty Oilfield Services Inc.,NYSE,17.00,"12,731,092",01/12/2018,216428564,Priced,Liberty Oilfield Services Inc. provides hydrau...,United States,Misc.,Energy,3.174335e-02,2
2,PACK,Ranpak Holdings Corp.,NYSE,10.00,"30,000,000",01/18/2018,300000000,Priced,Ranpak Holdings Corp. and its subsidiaries pro...,United States,Misc.,Consumer Cyclical,1.843263e-05,2
3,NINE,"Nine Energy Service, Inc.",NYSE,23.00,"7,000,000",01/19/2018,161000000,Priced,Nine Energy Service Inc. operates as an onshor...,United States,Misc.,Energy,2.670285e-01,2
4,ADT,ADT Inc.,NYSE,14.00,"105,000,000",01/19/2018,1470000000,Priced,ADT Inc. provides security and automation solu...,United States,Devices,Industrials,7.411440e-09,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,CASA,Casa Systems Inc,NASDAQ Global Select,13.00,"6,000,000",12/15/2017,78000000,Priced,Casa Systems Inc. provides software-centric br...,United States,Media,Technology,3.112301e-03,0
395,NMRK,"NEWMARK GROUP, INC.",NASDAQ Global Select,14.00,"20,000,000",12/15/2017,280000000,Priced,Newmark Group Inc. provides commercial real es...,United States,Misc.,Real Estate,2.921059e-08,2
396,TRVG,trivago N.V.,NASDAQ Global Select,11.00,"26,110,118",12/16/2016,287211298,Priced,trivago N.V. together with its subsidiaries op...,Germany,Search,Communication Services,7.067322e-01,0
397,AMTB,Amerant Bancorp Inc.,NASDAQ Global Select,13.00,"6,300,000",12/19/2018,81900000,Priced,Amerant Bancorp Inc. operates as the bank hold...,United States,Organization,Financial Services,3.483509e-06,1


### Merging Text Analysis with the Decision Tree Model

The first step in this final process is to split the data into their respective leaf nodes.

In [20]:
node1 = ipos_list.loc[(ipos_list['Offer Amount'] <= 395000000)]
node16 = ipos_list.loc[(ipos_list['Offer Amount'] > 395000000)]

node2 = node1.loc[(node1['Offer Amount'] <= 165000000)]
node9 = node1.loc[(node1['Offer Amount'] > 165000000)]

node17 = node16.loc[(node16['Group_Index'] <= 0.5)]
node22 = node16.loc[(node16['Group_Index'] > 0.5)] #Leaf

node3 = node2.loc[(node2['Offer Amount'] <= 59000000)]
node6 = node2.loc[(node2['Offer Amount'] > 59000000)]

node10 = node9.loc[(node9['Offer Amount'] <= 368000000)]
node13 = node9.loc[(node9['Offer Amount'] > 368000000)]

node18 = node17.loc[(node17['Offer Amount'] <= 3050000000)]
node21 = node17.loc[(node17['Offer Amount'] > 3050000000)] #Leaf

node4 = node3.loc[(node3['Group_Index'] <= 0.5)] #Leaf
node5 = node3.loc[(node3['Group_Index'] > 0.5)] #Leaf

node7 = node6.loc[(node6['Group_Index'] <= 1.5)] #Leaf
node8 = node6.loc[(node6['Group_Index'] > 1.5)] #Leaf

node11 = node10.loc[(node10['Offer Amount'] <= 247500000)] #Leaf
node12 = node10.loc[(node10['Offer Amount'] > 247500000)] #Leaf

node14 = node13.loc[(node13['Offer Amount'] <= 385000000)] #Leaf
node15 = node13.loc[(node13['Offer Amount'] > 385000000)] #Leaf

node19 = node18.loc[(node18['Offer Amount'] <= 2325000000)] #Leaf
node20 = node18.loc[(node18['Offer Amount'] > 2325000000)] #Leaf

print("Node 4:", len(node4))
print("Node 5:", len(node5))
print("Node 7:", len(node7))
print("Node 8:", len(node8))
print("Node 11:", len(node11))
print("Node 12:", len(node12))
print("Node 14:", len(node14))
print("Node 15:", len(node15))
print("Node 19:", len(node19))
print("Node 20:", len(node20))
print("Node 21:", len(node21))
print("Node 22:", len(node22))
x = len(node4)+len(node5)+len(node7)+len(node8)+len(node11)+len(node12)+len(node14)+len(node15)+len(node19)+len(node20)+len(node21)+len(node22)
x #check to see that we have all the data points - we do!

Node 4: 3
Node 5: 23
Node 7: 55
Node 8: 116
Node 11: 60
Node 12: 53
Node 14: 2
Node 15: 1
Node 19: 18
Node 20: 0
Node 21: 1
Node 22: 67


399

From here, we calculate the average probability of each of the leaf nodes to determine from where to extract our final shortlist.

In [408]:
print(node4['Google_prob'].sum(axis = 0, skipna = True)/len(node4))
print(node5['Google_prob'].sum(axis = 0, skipna = True)/len(node5))
print(node7['Google_prob'].sum(axis = 0, skipna = True)/len(node7))
print(node8['Google_prob'].sum(axis = 0, skipna = True)/len(node8)) #second highest average
print(node11['Google_prob'].sum(axis = 0, skipna = True)/len(node11))
print(node12['Google_prob'].sum(axis = 0, skipna = True)/len(node12))
print(node14['Google_prob'].sum(axis = 0, skipna = True)/len(node14))
print(node15['Google_prob'].sum(axis = 0, skipna = True)/len(node15)) #highest average
print(node19['Google_prob'].sum(axis = 0, skipna = True)/len(node19))
print(node20['Google_prob'].sum(axis = 0, skipna = True)) # no division as div/0 error would arise
print(node21['Google_prob'].sum(axis = 0, skipna = True)/len(node21))
print(node22['Google_prob'].sum(axis = 0, skipna = True)/len(node22))

0.07726789548907556
0.1796768292005607
0.12917560262859198
0.5073885141963883
0.25416867947540733
0.26188704148814895
0.18283347279845144
0.8221431942987762
0.1426279033968427
0.0
0.006879908932349326
0.16014592158957602


In [22]:
shortlist = node15
shortlist

,Symbol,Company Name,Exchange,Price,Shares,Date,Offer Amount,Actions,Description,Country,Group,Industry,Google_prob,Group_Index
256,TXG,"10X Genomics, Inc.",NASDAQ Global Select,39.0,"10,000,000",09/12/2019,390000000,Priced,10x Genomics Inc. a life science technology co...,United States,Business,Healthcare,0.822143,0


 Node 15 only has one company inside it, so let's grab the 9 best from the 2nd highest average node - Node 8.

In [25]:
short1 = node8.sort_values(by=['Google_prob'], ascending=False).head(9)
shortlist1 = shortlist.append(short1, ignore_index=True)
shortlist1

,Symbol,Company Name,Exchange,Price,Shares,Date,Offer Amount,Actions,Description,Country,Group,Industry,Google_prob,Group_Index
0,TXG,"10X Genomics, Inc.",NASDAQ Global Select,39.00,"10,000,000",09/12/2019,390000000,Priced,10x Genomics Inc. a life science technology co...,United States,Business,Healthcare,0.822143,0
1,ANAB,"ANAPTYSBIO, INC",NASDAQ Global Select,15.00,"5,000,000",01/26/2017,75000000,Priced,AnaptysBio Inc. a clinical stage biotechnology...,United States,Research,Healthcare,0.999989,2
2,IPHA,Innate Pharma SA,NASDAQ Global Select,5.50,"12,500,000",10/17/2019,68750000,Priced,Innate Pharma S.A. a biotechnology company dis...,France,Research,Healthcare,0.996670,2
3,SCPH,scPharmaceuticals Inc.,NASDAQ Global Select,14.00,"6,400,000",11/17/2017,89600000,Priced,scPharmaceuticals Inc. a pharmaceutical compan...,United States,Research,Healthcare,0.989601,2
4,KALA,"Kala Pharmaceuticals, Inc.",NASDAQ Global Select,15.00,"6,000,000",07/20/2017,90000000,Priced,Kala Pharmaceuticals Inc. a biopharmaceutical ...,United States,Research,Healthcare,0.985240,2
5,CABA,"Cabaletta Bio, Inc.",NASDAQ Global Select,11.00,"6,800,000",10/25/2019,74800000,Priced,Cabaletta Bio Inc. a clinical-stage biotechnol...,United States,Research,Healthcare,0.983492,2
6,ERYP,Erytech Pharma S.A.,NASDAQ Global Select,23.26,"4,686,106",11/10/2017,108998826,Priced,ERYTECH Pharma S.A. a clinical-stage biopharma...,France,Research,Healthcare,0.977425,2
7,DTIL,PRECISION BIOSCIENCES INC,NASDAQ Global Select,16.00,"7,900,000",03/28/2019,126400000,Priced,Precision BioSciences Inc. operates as a genom...,United States,Research,Healthcare,0.976034,2
8,SLDB,Solid Biosciences Inc.,NASDAQ Global Select,16.00,"7,812,500",01/26/2018,125000000,Priced,Solid Biosciences Inc. a life science company ...,United States,Research,Healthcare,0.973965,2
9,MRSN,"Mersana Therapeutics, Inc.",NASDAQ Global Select,15.00,"5,000,000",06/28/2017,75000000,Priced,Mersana Therapeutics Inc. a clinical stage bio...,United States,Research,Healthcare,0.973725,2


In [414]:
shortlist1.to_csv('Final Company Shortlist.csv')

### Results

The results outline the 10 most likely acquisition candidates to all be some form of healthcare firm - mostly in the biotechnology space. This could potentially be attributed to Google's earlier acquisition of Verily that has now formed their life sciences research division. In contrast to the other big tech companies that were compared to Alphabet (Facebook, Apple, etc.), Google was likely the only firm with a life sciences acquisition in that dataset.

### Identified Risks (there are a lot more of them)

- The final decision tree only implemented two criterion to partition the data: price and functional group. While this makes the model simple to interpret, it runs the risk of not being robust.

- The initial regression approach did not work as a result of the lack of correlation between the jumps in stock price and the aforementioned independent variables. This could signal a lack of validity in using the independent variables in the other analysis measures.

- More quantitative measures could have been used to pair with the categorical and text-based data points (e.g EPS, financial ratios).

### Concluding Remarks 

This project examined which companies would be most likely to be acquired by Google's parent company Alphabet from a list of firms that went public between January 1st, 2015, and December 31st, 2019. The initial hypothesis suggested that the result would likely point in the direction of Google's 'other bets' groups, where a company focused on some form of deep research or innovation would most likely be acquired to complement Google's existing and growing enterprises on these fronts.

A simple regression was first run, but the low R^2 of only ~20% suggested that this model should not be used. A logistic regression was also tested, but the model proved unreliable and was also abandoned. However, the decision tree model was able to separate the data into 12 (11) distinct groups, from which point it was combined with the text-based analysis to determine that 10X Genomics Inc., A California-based biotechnology company, would be the most likely to be acquired by Alphabet. This result is in line with the initial hypothesis, as Google has a life sciences research division as part of their 'other bets' group. 

The model itself does presents some risks, particularly with respect to the quantity and quality of the data that was used, and these aspects are what would need to be improved on as a next step from what this project's model predicted.